# Debugging with Evidently Test Suites and Reports

In [1]:
import datetime
import pandas as pd

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

from evidently.test_suite import TestSuite
from evidently.test_preset import DataDriftTestPreset, DataQualityTestPreset

from joblib import dump, load
from optbinning import Scorecard

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

/home/fini/.local/share/virtualenvs/05-monitoring-NIVV9OLH/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/fini/.local/share/virtualenvs/05-monitoring-NIVV9OLH/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/fini/.local/shar

(CVXPY) Jun 28 04:42:35 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jun 28 04:42:35 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')


In [2]:
t = pd.read_csv("/home/fini/github-projects/mlops/data/heloc_dataset_v1.csv")

In [3]:
{
    col.lower(): col for col in t.columns
}

{'riskperformance': 'RiskPerformance',
 'externalriskestimate': 'ExternalRiskEstimate',
 'msinceoldesttradeopen': 'MSinceOldestTradeOpen',
 'msincemostrecenttradeopen': 'MSinceMostRecentTradeOpen',
 'averageminfile': 'AverageMInFile',
 'numsatisfactorytrades': 'NumSatisfactoryTrades',
 'numtrades60ever2derogpubrec': 'NumTrades60Ever2DerogPubRec',
 'numtrades90ever2derogpubrec': 'NumTrades90Ever2DerogPubRec',
 'percenttradesneverdelq': 'PercentTradesNeverDelq',
 'msincemostrecentdelq': 'MSinceMostRecentDelq',
 'maxdelq2publicreclast12m': 'MaxDelq2PublicRecLast12M',
 'maxdelqever': 'MaxDelqEver',
 'numtotaltrades': 'NumTotalTrades',
 'numtradesopeninlast12m': 'NumTradesOpeninLast12M',
 'percentinstalltrades': 'PercentInstallTrades',
 'msincemostrecentinqexcl7days': 'MSinceMostRecentInqexcl7days',
 'numinqlast6m': 'NumInqLast6M',
 'numinqlast6mexcl7days': 'NumInqLast6Mexcl7days',
 'netfractionrevolvingburden': 'NetFractionRevolvingBurden',
 'netfractioninstallburden': 'NetFractionInstallB

In [3]:
tt = t.tail(10).copy()

In [4]:
tt["RiskPerformance"] = (tt["RiskPerformance"]=="Bad").astype("int8")

In [50]:
r = {}
for i, j in enumerate(tt.rename(columns=str.lower).to_dict(orient="records")):
    r[f"record-{i}"] = j

r

{'record-0': {'riskperformance': 0,
  'externalriskestimate': 57,
  'msinceoldesttradeopen': 176,
  'msincemostrecenttradeopen': 4,
  'averageminfile': 87,
  'numsatisfactorytrades': 18,
  'numtrades60ever2derogpubrec': 0,
  'numtrades90ever2derogpubrec': 0,
  'percenttradesneverdelq': 89,
  'msincemostrecentdelq': 2,
  'maxdelq2publicreclast12m': 4,
  'maxdelqever': 6,
  'numtotaltrades': 18,
  'numtradesopeninlast12m': 1,
  'percentinstalltrades': 56,
  'msincemostrecentinqexcl7days': 4,
  'numinqlast6m': 1,
  'numinqlast6mexcl7days': 1,
  'netfractionrevolvingburden': 93,
  'netfractioninstallburden': 72,
  'numrevolvingtradeswbalance': 5,
  'numinstalltradeswbalance': 3,
  'numbank2natltradeswhighutilization': 1,
  'percenttradeswbalance': 100},
 'record-1': {'riskperformance': 0,
  'externalriskestimate': 84,
  'msinceoldesttradeopen': 313,
  'msincemostrecenttradeopen': 6,
  'averageminfile': 103,
  'numsatisfactorytrades': 25,
  'numtrades60ever2derogpubrec': 0,
  'numtrades90ev

## Load data and model

In [6]:
ref_data = pd.read_parquet('data/reference.parquet')
current_data = pd.read_parquet('data/raw.parquet')

In [36]:
current_data.fillna(0).astype("int16")

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'datetime.date'

In [3]:
model = Scorecard.load('models/scorecard-model.pkl')

In [8]:
# data labeling
TARGET = "RiskPerformance"
num_features = [
            "AverageMInFile",
            "MSinceMostRecentInqexcl7days",
            "PercentTradesNeverDelq",
            "ExternalRiskEstimate",
            "NetFractionRevolvingBurden",
            "NumSatisfactoryTrades",
            "PercentInstallTrades"
      ]
cat_features = []

In [9]:
t = ref_data[num_features]

In [10]:
t.head(1).to_dict(orient="records")[0]

{'AverageMInFile': 76,
 'MSinceMostRecentInqexcl7days': 0,
 'PercentTradesNeverDelq': 100,
 'ExternalRiskEstimate': 69,
 'NetFractionRevolvingBurden': 12,
 'NumSatisfactoryTrades': 36,
 'PercentInstallTrades': 25}

In [11]:
rc = {'AverageMInFile': 76,
 'MSinceMostRecentInqexcl7days': 0,
 'PercentTradesNeverDelq': 100,
 'ExternalRiskEstimate': 69,
 'NetFractionRevolvingBurden': 12,
 'NumSatisfactoryTrades': 36,
 'PercentInstallTrades': 25}

In [12]:
pd.DataFrame([rc])

,AverageMInFile,MSinceMostRecentInqexcl7days,PercentTradesNeverDelq,ExternalRiskEstimate,NetFractionRevolvingBurden,NumSatisfactoryTrades,PercentInstallTrades
0,76,0,100,69,12,36,25


In [16]:
model.score(t.head(1).to_numpy().reshape(-1,1))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

Problematic data we created was marked for the October 1 to 12. We can pretend that we found that date from monitoring.

In [ ]:
problematic_data = current_data.loc[
    (current_data["operation_date"] >= datetime.date(2023,10,1)) & 
    (current_data["operation_date"]< datetime.date(2023,10,13))
    ][current_data.columns[::-1]]

In [ ]:
problematic_data.head()

## Generate Test Suite and Report

In [ ]:
column_mapping = ColumnMapping(
    prediction='prediction',
    numerical_features=num_features,
    categorical_features=cat_features,
    target=None
)

In [ ]:
problematic_data['prediction'] = model.score(problematic_data[num_features + cat_features].fillna(0))
ref_data['prediction'] = model.score(ref_data[num_features + cat_features].fillna(0))

In [ ]:
test_suite = TestSuite(tests = [DataDriftTestPreset()])
test_suite.run(reference_data=ref_data, current_data=problematic_data, column_mapping=column_mapping)

In [ ]:
test_suite.show(mode='inline')

Use report to support analysis. Instead of using the metrics individually we use DataDriftPreset

In [ ]:
report = Report(metrics = [DataDriftPreset()])
report.run(reference_data=ref_data, current_data=problematic_data, column_mapping=column_mapping)

In [ ]:
report.show(mode='inline')

In [ ]:
data_quality_test_suite = TestSuite(tests=[
   DataQualityTestPreset(),
])
 
data_quality_test_suite.run(reference_data=ref_data, current_data=problematic_data, column_mapping=column_mapping)
data_quality_test_suite

Check multiple tests

In [ ]:
from evidently.tests import TestColumnDrift, TestShareOfDriftedColumns

Test type and applicability
- z: categorical
- g_test: categorical

In [ ]:
test_types = ["jensenshannon", "kl_div", "jensenshannon", "cramer_von_mises", "psi", "anderson", "anderson"]
per_column_stattest = dict(zip(num_features, test_types))

In [ ]:
data_drift_dataset_tests = TestSuite(tests=[
    TestShareOfDriftedColumns(per_column_stattest=per_column_stattest),
])

data_drift_dataset_tests.run(reference_data=ref_data, current_data=problematic_data)
data_drift_dataset_tests

In [ ]:
# multiple_report = Report(metrics = [DataDriftPreset(), DataQualityTestPreset()])
# TestSuite(tests = [DataDriftTestPreset()])
# multiple_report.run(reference_data=ref_data, current_data=problematic_data, column_mapping=column_mapping)

Another way to set tests

In [ ]:

data_drift_dataset_tests = TestSuite(tests=[
    TestShareOfDriftedColumns(num_stattest='psi', cat_stattest='jensenshannon'),
])

data_drift_dataset_tests.run(reference_data=ref_data, current_data=problematic_data)
data_drift_dataset_tests



In [ ]:
multiple_test = TestSuite(tests = [DataDriftTestPreset(), DataQualityTestPreset()])
multiple_test.run(reference_data=ref_data, current_data=problematic_data, column_mapping=column_mapping)

In [ ]:
multiple_test